In [27]:
import sqlite3
import pandas as pd
import yaml

In [3]:
con = sqlite3.connect("example.db")

In [4]:
cursor = con.cursor()

In [ ]:
def init_db(conn, cursor):
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id TEXT,
        type TEXT CHECK(type IN ('income', 'expense')),
        category TEXT,
        amount REAL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    conn.commit()

In [7]:
init_db(conn=con, cursor=cursor)

### Первый подход на примере использования тяжелых моделей

In [5]:
import ollama


client = ollama.Client()


model = "gemma3:1b"
prompt = "Что такое Python?"


response = client.generate(model=model, prompt=prompt)

print("Response from Ollama:")
print(response.response)

Response from Ollama:
Python — это высокоуровневый, интерпретируемый, объектно-ориентированный язык программирования. Он разработан в конце 1980-х годов и активно используется в качестве языка программирования для различных целей.  Вот более подробное объяснение:

**1. Основные характеристики Python:**

* **Читаемость:** Python известен своей четкой и лаконичной синтаксисом, что делает его относительно легким для понимания и чтения кода. Он похож на английский язык, что позволяет использовать его для понимания кода без глубокого знания программирования.
* **Универсальность:** Python можно использовать для широкого спектра задач, от разработки веб-сайтов и приложений до анализа данных, машинного обучения и научных вычислений.
* **Динамическая типизация:** Типы переменных определяются во время выполнения, а не во время компиляции. Это упрощает разработку, но может привести к ошибкам, если типы не проверяются правильно.
* **Интерпретируемый:**  Код Python выполняется построчно интерпретат

In [68]:
def execute_sql(sql):
    try:
        cursor.execute(sql)
        con.commit()
        return "Успешно выполнено"
    except Exception as e:
        return f"Ошибка: {e}"

In [26]:
# prompt = f"""
#     У тебя есть таблица transactions со следующими столбцами:
#     id, user_id, type (income/expense), category, amount, date, note.
#     income - означает доход
#     expense - означает трата

#     вот пример создания таблицы:

#     CREATE TABLE IF NOT EXISTS transactions (
#         id INTEGER PRIMARY KEY AUTOINCREMENT,
#         user_id TEXT,
#         type TEXT CHECK(type IN ('income', 'expense')),
#         category TEXT,
#         amount REAL,
#         date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
#     )

#     Пользователь с ID {user_id} дал команду:
#     "{natural_language_input}"

#     Сгенерируй корректный SQL-запрос на SQLite для выполнения этого действия.
#     Ответь только SQL-запросом, без пояснений.

#     Всегда используй SQLite-синтаксис. Не добавляй пояснений, только SQL-запрос.

#     Пример 1:
#     Пользователь с ID user_1 дал команду:
#     "Добавь трату 500 рублей на еду вчера"
#     Ответ:
#     INSERT INTO transactions (user_id, type, category, amount, date)
#     VALUES ('user_1', 'expense', 'еда', 500, date('now', '-1 day'));

#     Пример 2:
#     Пользователь с ID user_2 дал команду:
#     "Я заработал 1500 рублей с фриланса"
#     Ответ:
#     INSERT INTO transactions (user_id, type, category, amount)
#     VALUES ('user_2', 'income', 'фриланс', 1500);

#     Пример 3:
#     Пользователь с ID user_3 дал команду:
#     "Покажи все мои траты за последнюю неделю"
#     Ответ:
#     SELECT * FROM transactions
#     WHERE user_id = 'user_3' AND type = 'expense' AND date >= date('now', '-7 day');

#     Пример 4:
#     Пользователь с ID user_4 дал команду:
#     "Измени сумму последней траты на транспорт до 300"
#     Ответ:
#     UPDATE transactions
#     SET amount = 300
#     WHERE user_id = 'user_4' AND category = 'транспорт' AND type = 'expense'
#     ORDER BY date DESC
#     LIMIT 1;

#     Пример 5:
#     Пользователь с ID user_5 дал команду:
#     "Удалить последний доход"
#     Ответ:
#     DELETE FROM transactions
#     WHERE user_id = 'user_5' AND type = 'income'
#     ORDER BY date DESC
#     LIMIT 1;
#     """

In [32]:
def load_prompts(system_path="../prompts/v1/system.yaml", user_path="../prompts/v1/user.yaml"):
    with open(system_path, encoding="utf-8") as f:
        system_prompt = yaml.safe_load(f)["system_prompt"]

    with open(user_path, encoding="utf-8") as f:
        user_prompt_template = yaml.safe_load(f)["user_prompt_template"]

    return system_prompt, user_prompt_template

In [52]:
from pydantic import BaseModel, field_validator
import re

class SqlResponse(BaseModel):
    sql: str

    @field_validator("sql", mode="before")
    @classmethod
    def clean_sql_block(cls, value: str) -> str:
        # Удаляем обёртки ```sql ... ```
        value = re.sub(r"```sql|```", "", value, flags=re.IGNORECASE).strip()
        return value

    @field_validator("sql")
    @classmethod
    def must_look_like_sql(cls, value: str) -> str:
        if not re.match(r"^(SELECT|INSERT|UPDATE|DELETE)", value.strip(), re.IGNORECASE):
            raise ValueError("Ответ не похож на SQL-запрос.")
        return value.strip()

In [56]:
def generate_sql(client, model, natural_language_input, user_id="user_1"):
    system_prompt, user_prompt_template = load_prompts()

    user_prompt = user_prompt_template.format(user_id=user_id, input_text=natural_language_input)

    response = client.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    )

    # raw_text = response["message"]["content"]
    # try:
    #     parsed = SqlResponse(sql=raw_text)
    #     return parsed.sql  # чистый SQL
    # except Exception as e:
    #     print(f"❌ Ошибка парсинга ответа: {e}")
    #     print(f"🔽 Сырой ответ:\n{raw_text}")
    #     return None

    return response

In [ ]:
sql_response = generate_sql(client=client, model=model, natural_language_input="Добавь трату 879 рублей в макдональдс позавчера")

In [66]:
print(sql_response["message"]["content"])

INSERT INTO transactions (user_id, type, category, amount, date)
VALUES ('user_1', 'expense', 'макдональдс', 879, date('now', '-1 day'));


In [69]:
execute_sql(sql_response["message"]["content"])

'Успешно выполнено'